In [1]:
import pandas as pd
import numpy as np

In [2]:
def h_prima(H: int, mensaje: str) -> str:
    
    # Se divide el h0 en 4 partes de la siguiente forma
    mascara = 0xFFFFFFFF
    a0 = (H & (mascara << 96)) >> 96
    b0 = (H & (mascara << 64)) >> 64
    c0 = (H & (mascara << 32)) >> 32
    d0 = H & mascara
    
    # Se especifica los shifts por ronda
    s = []
    s[0:15] =  [ 7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22 ]
    s[16:31] = [ 5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20 ]
    s[32:47] = [ 4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23 ]
    s[48:63] = [ 6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21 ]
      
    k = []
    for i in range(64):
        k.append(int(np.floor(2**32*abs(np.sin(i + 1)))) & mascara)
    
    # Se inicializan las variables
    A = a0
    B = b0
    C = c0
    D = d0
    
    # Se divide el mensaje en chunks de 32 bits
    M = []
    for g in range(0,len(mensaje),4):
        M.append(int.from_bytes(mensaje[g:g+4], byteorder='little'))
    
    for j in range(64):
        F,g = 0,0
        if 0 <= j <= 15:
            F = (B & C) | ((~ B) & D)
            g = j
        elif 16 <= j <= 31:
            F = (D & B) | ((~ D) & C)
            g = (5*j + 1) % 16
        elif 32 <= j <= 47:
            F = B ^ C ^ D
            g = (3*j + 5) % 16
        elif 48 <= j <= 63:
            F = C ^ (B | (~ D))
            g = (7*j) % 16
        
        F = (F + A + k[j] + M[g]) & mascara
        A = D
        D = C
        C = B
        B = (B + (F << s[j] | F >> (32-s[j])) & mascara) & mascara
        
    a0 = (a0 + A) & mascara
    b0 = (b0 + B) & mascara
    c0 = (c0 + C) & mascara
    d0 = (d0 + D) & mascara

    return a0 + (b0 << 32) + (c0 << 64) + (d0 << 96)

In [3]:
def md5_to_hex(digest):
    raw = digest.to_bytes(16, byteorder='little')
    return '{:032x}'.format(int.from_bytes(raw, byteorder='big'))

In [4]:
def custom_md5(m: str, h0: int) -> str:
    # Argumentos:
    #  m: str - mensaje
    # h0: int - constante inicial H_0
    # Retorna:
    #  str - hash MD5 correcto del mensaje en formato hexadecimal
    
    # lo primero es dejar el mensaje divisible por 512
    
    # Ahora se agregarán un uno y ceros hasta que quede divisible por 512
        
    # Se agrega el largo original del mensaje
    message = bytearray(m, 'utf-8') #copy our input into a mutable buffer
    orig_len_in_bits = (8 * len(message)) & 0xffffffffffffffff
    message.append(0x80)
    while len(message)%64 != 56:
        message.append(0)
    message += orig_len_in_bits.to_bytes(8, byteorder='little')

        
    # Ahora se toman los estados para calcular el h_prima
    H = h0
    for i in range(0,len(message),64):
        H = h_prima(H, message[i: i + 64])
        
    return md5_to_hex(H)   

In [5]:
df = pd.read_csv("mensajes_p3.csv", header=None, names=["indice", "mensajes"])
count = 0
mensajes = []
while custom_md5("fcjimenez@uc.cl", 16207084 * 100 + count ) in df["indice"].unique():
    mensajes.append(df[df["indice"] == custom_md5("fcjimenez@uc.cl", 16207084 * 100 + count )]["mensajes"].item())
    count+=1

In [6]:
len(mensajes)

200

In [7]:
def binary_to_text(binary):
    texto = ""
    numeros = []
    for i in range(0, len(binary), 8):
        caracter = int(binary[i: i+8],2)
        numeros.append(caracter)
        caracter = chr(caracter)
        texto += caracter
    return texto, numeros

In [8]:
def propio_xor(m1, m2):
    res = ""
    for i in range(len(m1)):
        if m1[i] == m2[i]:
            res += "0"
        else:
            res += "1"
    return res

In [9]:
def probable_space_count_vector(cyphertext, mensajes):
    length = int(len(cyphertext)/8)
    counts= [0]*length
    
    for c in mensajes:
        messages_xor = propio_xor(c, cyphertext)
        _, numeros = binary_to_text(messages_xor)
        for i in range(len(numeros)):
            if numeros[i] > 64:
                counts[i] += 1
    return [round(c/ len(mensajes), 4) for c in counts]

In [10]:
def max_index(i, lista):
    indice = 0
    max_value = 0
    for j in range(len(lista)):
        if lista[j][i] > max_value:
            indice = j
            max_value = lista[j][i]
    return indice

In [11]:
def pertenecer_grupo(msj, grupo):
    cuantos = 0
    num_por_msj = len(msj)/8
    for integrante in grupo:
        res = propio_xor(msj, integrante)
        _, nums = binary_to_text(res)
        for num in nums:
            if 0 <= num <= 31 or 65 <= num <= 90 or 97:
                cuantos += 1
    return cuantos / (len(grupo)*num_por_msj)

In [12]:
def chosen(mensaje, idx):
    contador = 0
    for i in range(0,len(mensaje),8):
        if contador == idx:
            return mensaje[i:i+8]
        contador += 1

In [54]:
def break_random_otp(encrypted_messages: list) -> list :
    # Argumentos:
    #  encrypted_messages: list[str] - lista de mensajes encriptados.
    # Retorna:
    # list[str] - lista de mensajes decriptados
    
    # Creo un diccionario con llave el mensaje y value un booleano que indica si está en un grupo de mensajes o no
    agrupado = dict()
    resultados = dict()
    for m in encrypted_messages:
        agrupado[m] = False
        resultados[m] = ""
    # Luego reviso todos los mensajes y asigno grupo a los que pueda siempre y cuando no exceda los 15 mensajes
    # por grupo y que la cantidad de xor entre el mensaje que se recorre y el que se compara tenga la misma cantidad que
    # el largo del mensaje, es decir, que el xor que revisamos que un espacio con letras minusculas esté en el rango
    # en los 10 caracteres
    elegidos = []
    for m in encrypted_messages:
        grupo = []
        pase = False
        for m2 in encrypted_messages:
            if m2 != m and not agrupado[m2] and not agrupado[m]:
                resultado = propio_xor(m, m2)
                text, numbers = binary_to_text(resultado)
                cuantos = 0
                for num in numbers:
                    if 0 <= num <= 31 or 65 <= num <= 90:
                        cuantos += 1
                if cuantos == (len(m)/8) and len(grupo) <= 13:
                    grupo.append(m2)
                    pase = True
                    agrupado[m2] = True
        if pase:
            agrupado[m] = True
            grupo.append(m)
            elegidos.append(grupo)
    
    # Luego a cada mensaje que no se le ha asignado un grupo, se le asigna uno
#     print("elegidos: ", elegidos)
    for llave in agrupado:
        if not agrupado[llave]:
            chosen_one = 0
            indices_values = dict()
            for j in range(len(elegidos)):
                indices_values[j] = pertenecer_grupo(llave, elegidos[j])
            indices = sorted(indices_values.items(), key=lambda item: item[1], reverse=True)
#             print("indices: ", indices)
            if len(list(filter(lambda x: len(x) < 15, elegidos))) > 0:
                for tupla in indices:
                    if len(elegidos[tupla[0]]) < 15:
                        elegidos[tupla[0]].append(llave)
                        agrupado[llave] = True
                        break
            else:
                elegidos[indices[0][0]].append(llave)
                agrupado[llave] = True
                
    # Revisa que no haya grupos con menos de 15 mensajes, en caso que haya los reasigna de acuerdo a la probabilidad
    # de que pertenezcan allí. Esta probabilidad se calcula como que corresponda al xor entre una letra minúscula y/o un espacio
    # entre todos los mensajes para un mensaje, es decir, casos favorables de xor con el total de mensaje de dicho grupo
#     while len(list(filter(lambda x: len(x) < 15, elegidos))) > 0:
#         elegidos.sort(key = len)
#         minimo = elegidos.pop(0)
#         for msj in minimo:
#             indices_values = dict()
#             for j in range(len(elegidos)):
#                 indices_values[j] = pertenecer_grupo(msj, elegidos[j])
#             indices = sorted(indices_values.items(), key=lambda item: item[1], reverse=True)
# #             print("indices: ", indices)
#             if len(list(filter(lambda x: len(x) < 15, elegidos))) > 0:
#                 for tupla in indices:
#                     if len(elegidos[tupla[0]]) < 15:
#                         elegidos[tupla[0]].append(msj)
#                         break
#             else:
#                 elegidos[indices[0][0]].append(msj)
    
    # Luego se recorre cada grupo y se calcula la probabilidad de que haya un espacio, se crea la posible llave a partir 
    # de este resultado y se realiza el xor para decriptar cada mensaje
#     print(resultados)
    for grupo in elegidos:
        listeilor = []
        for m in grupo:
            a = probable_space_count_vector(m, grupo)
            listeilor.append(a)
        max_indices = [max_index(i, listeilor) for i in range(len(listeilor[0]))]
        encrypted_spaces = ""
        for i in range(len(max_indices)):
            encrypted_spaces += chosen(grupo[max_indices[i]],i)
        space = "00100000"*int(len(grupo[0])/8)
        probable_key = propio_xor(encrypted_spaces,space)
        for m in grupo:
            dec = propio_xor(m, probable_key)
            text, _ = binary_to_text(dec)
            resultados[m] = text
            
    return resultados

In [55]:
mensa = ['\x02\x08\x08Ir_\x1f}\x0fT',
 'jxB\x16jLW\x01=!',
 '@An^^\x1c1a(6',
 '\x04pZh\\g\x7fw)f',
 ']Z+H\x1f\x03-4-y',
 '\x02\x08\x0cOeO\x17<\x14O',
 '\x18&t+ &\x0c\x15{F',
 '<\x13l%`~\\up"',
 ';aEcxd4\x0ca<',
 'zio\x1fM7$/(D',
 'h\\u`4uX;\x1d"',
 'x\x13lq4wU9\x1d+',
 '\x0eN\x11DrY\\</R',
 'hnB\x0fp\x0fOT+d',
 '#%E$6X{\x18/+',
 'i\x08"d`6U0\\0',
 '"\x16\x16r\x10\x07\x06\x13B2',
 '4|\x05m\x01R.Y".',
 'vOT2X)y\x08Sa',
 ')(J?=S/Y#*',
 'D\\g@\x1b\x1e~`2s',
 "]\x12{jzs\x19 S'",
 'G\\\x10BeORk\x1eO',
 '|\x1c\x031C.hO\x1dN',
 'z{C(xN/\x0b &',
 'R[G?+}ee3m',
 '\x06X\x1dU7]\x1au\x18U',
 '\x1atLhi|~o8g',
 '0=\x15\x0bq\x04\x07\x15iw',
 '\x0fMXDbH\x1b\x7f\x17X',
 'io\x03\x05l\x02@\x180!',
 'a\x0cQ&X+s\rX(',
 'Y:byh=\x14PAH',
 '\x06}Mh|f~h8)',
 '4(J!4\x1d?\x16a&',
 'n\x0fgir6L%\x13c',
 'RT\x08;~jua3)',
 'z7:D\x107\x1e,aX',
 '\x04\x08\x0cRuORs\x15\x1d',
 'p|\x00\x03wLA\x06&o',
 'h<B#k\x08\x07\x00!d',
 'GG\x1e\x07cB\x17<\x08M',
 '\\\nU6^jb\tRq',
 '+\x10U<\x10\x11\x0c\x1fR$',
 "^5K'f|p`8'",
 ']Y{I\x1aZ*{=s',
 'kA\x03\x1bIjb\x15Ri',
 'r\x08"i}x\\y\x1d!',
 'yn\x11\x07vM\x07><r',
 '~x\x16\x16`\x1e\x0bT*n',
 '{\x7f,K\x06r\x16/$T',
 '?m\x0b !\x1d0\x17$*',
 'imB\x15l\x18OZiO',
 "\x177'h'<\x17Q\x12\x08",
 '2(F(x\\<\x18(!',
 'u\nkiqq\\uR%',
 'qo\x03\x06`@\x07\x00!`',
 '\x1drbm. \x18\\W\x08',
 "eOG:_'p\x18\x1c&",
 'usLBQ\x04F\x00:!',
 "z&0]U75'/T",
 '.(B+xD4\x0ca8',
 "\x0c&'|!=\x13\x15@\x03",
 '426[\x00v\x1c##\\',
 "<-0ETc\x1f'aD",
 '/\x1d\x10r\x04D\x07\x15Da',
 '2\x18K:@/1\x15Uc',
 'h=\x16\nl\x02LT>i',
 '2\x06M X8d\x02It',
 '9+q\x085y\x13b.^',
 ')\x06\n<\x0c\x02O\x08_$',
 'z09\x08\x15yW*(C',
 '@An\x0c\x1a\x0f-`zp',
 "}\t\x03'D/1\x04Dc",
 '\x13a[hikep8{',
 '3,\x7f@\x15y\x131aE',
 '\x0e3i\x7fh=\x14\x15A\x03',
 'l\x06\x10{\x0bD\x18\x14^"',
 'ht\x0c\x05%*H\x06=s',
 '\x08NXO~YRx\x1eN',
 '<\x1akb|bP;Zc',
 'w\x1cPs^+x\x12Xb',
 'sjB\x16w\x15\x07\x15.`',
 '\x0b!s+"&\x12[FF',
 '$\x10Yx\x06\x01\x1fP\x174',
 '=`_(6X?Y)*',
 '\x1az\\hxyte))',
 'R{G<+zc}4g',
 '\x18\t\\E\x10\t*{46',
 '\x15;\x08\x04dytvq)',
 '\x1e7uxh<\x15QW\x14',
 '}nB5l\x02T\x00&o',
 '\x14Z{I\x1f\x11)f3b',
 '?(R"-\x1d?\x16(!',
 '2\x1dF I$e\x0cXh',
 '\x117foh(\x15Q\x12\t',
 'z+7MTd\x1f0$G',
 'y\x0e"c{dM,\x10%',
 'Rz]<+o}h}f',
 "RbI<hf1i8'",
 "2\x7f\x08\x06U7.'2\x1c",
 'lGI< \x0c\x0e\x0cC$',
 'FB+_\n\x1b,`?r',
 '\x03QXA{E\x02l\x1eY',
 'R[dA^\x0f*`?d',
 '3,7\x08\x02x\x1e!$\x10',
 'A]c\\\x17\x1f=qv6',
 'fN\x03\x12\x0c9x\x0fZj',
 '}y\x06\x07aLB\x1a*n',
 '"U\x1as\x16\x08\x0b\\G3',
 "?\x7f;G\x17b\x1a'/D",
 ')60FTD\x1a+5X',
 'FFf\x0c\x17\x0e-47y',
 '<\x08m`g8\x19\x02Xc',
 '\tL\x1dUd\n\x1dz[M',
 '!\x10Yu\rD\x1c\x19A$',
 '\x13@\x1d\x07eC\x15t\x0f\x10',
 '5}Ym;U2\x15%=',
 "\x1a+'i-'\x0f\x15]\x10",
 'Rt\\hnobaq)',
 "?3:[\x17e\x12'/\x1e",
 '\x180h}-i\x13P@F',
 '=(B9v\x1d\x0c\x185,',
 'l\x01\x1cp\x06\x17\x0c\x0eR$',
 '\x7f\x0eJ=I.1\x02Rk',
 '\x143u`)+\x17P\x12\x08',
 'G\\\x10B7X\x1b{\x13I',
 ']Gl\x0c\t\x12;zz~',
 '\x17t[-*.Hk()',
 '\x0c\x06Xny\n\x06t\x1e\x1d',
 '\x02I\x13PeC\x06yW\x1d',
 '?f\x05m\x16R,Y- ',
 'hi\x07\x10)LT\x1c,!',
 'r=\x16\n`La\x18&`',
 '\x10I\x14Kd\n\x1dz[I',
 "\x1c 'j&-[AG\x05",
 '4|\x0b".X)Y (',
 '%\x1a\x15y\r\x10O\x10B/',
 'hR"Wq{\\8_&',
 '\x11tFhoa1f8}',
 'p\x10"nqsIu[*',
 'l\x01\x11yC\n\n\x1dE/',
 '2\x07F!\x0c+c\x0cN&',
 ";f_m,RwY2'",
 'UJgI\rT~@2s',
 "b\x03F'I&hATh",
 '2!F%I81\x12Ui',
 '}iB\x16m\t^T!`',
 '\x1a7a~$%\x02\x19\x12\x04',
 'RyG?n|=$-e',
 'zA](xU:\x1da)',
 'UGo\x0c\x0e\x0f*45x',
 '\x01d])o"1e3m',
 '?(J)<X?Y <',
 'z+7I\x007\x18!"Q',
 'u\ng%}e\x19%X1',
 ')U\x1fu\x11\x17\x1b\\C(',
 '\x1c3se-:\x08\x15S\x08',
 '59\x7f\\\x1crW$ \\',
 '\x06pZh\x7ffpj}}',
 'GG\rS7E\x14<\x0fU',
 'Y\x06on:,W\x15Q\t',
 '$U\x17s\x17D\n\nR/',
 'i\x1fjlzq\x19=T0',
 '\x14Ab_^\t.q9b',
 '>\x14\x15<\x1a\x01\x0e\x0eDo',
 '{\x1c\x035M)tAOc',
 'Q_nB\nZ6}76',
 '\x1at\\f+W~q/l',
 '(\x10\x1d<\n\nO\x08X4',
 '\x14E\x19K{\n\x11e\x17T',
 ";-&\x08\x04e\x18-'\x10",
 'aO@<Y&uAZo',
 'wOE?E)z\x04O&',
 '<n\x03\x0bi\x03U\x07ih',
 'Q\t\x7fC\t\x1b,p)6',
 '\x02yM)xk0$\ta',
 ')69A\x17v\x03+.^',
 'Z\tcI^\x0f0f5z',
 '\x0fI\x16C7Y\x1bx\x1e\x1d',
 '\\@f\x00^\x182q-6',
 'n\\mpf6[:D0',
 '8\x1a\x1coC\x13\x06\x08_a',
 '\x14 fo-:Z\x15f\x0e',
 'XLo\x0c\x1f\x14:49z',
 '>UM<4\r\x1b\x14\x175',
 'u\x12"q|s\x193O,',
 "'\x1b\x1cy\x10D\x1a\x12U$",
 "\x16&'x)0[R@\x07",
 ".0-A\x17v\x1bb'Q",
 'z\x19aqxo\x196\\3',
 "\x164'c-;[S[\x08",
 '}\x1en`4y_uI,',
 't(b xI3\x103;',
 '5c\x05m\x0bU>Y#*',
 '"\x01U<\x05\x0b\x1d\\C)',
 '}\ng%`~\\uM1',
 'd\n\x03*C?1\x00Jg',
 '\x127c+<!\x1e\x15T\x0f',
 'G@\x19C7K\x1en\x1e\\']

In [56]:
a = break_random_otp(mensajes)

In [57]:
len(a)

200

In [58]:
a

{'01111010001101110011101001000100000100000011011100011110001011000110000101011000': ' held in h',
 '00110011001011000111111101000000000101010111100100010011001100010110000101000101': 'is hands u',
 '00110100001100100011011001011011000000000111011000011100001000110010001101011100': 'nmistakabl',
 '00111111011111110011101101000111000101110110001000011010001001110010111101000100': 'e document',
 '00111011001011010010011000001000000001000110010100011000001011010010011100010000': 'ary proof ',
 '00110101001110010111111101011100000111000111001001010111001001000010000001011100': 'of the fal',
 '00101001001101100011100101000001000101110111011000000011001010110010111001011110': 'sification',
 '01111010001100000011100100001000000101010111100101010111001010100010100001000011': ' of an his',
 '00101110001100000010110101000001000101110111011000011011011000100010011101010001': 'torical fa',
 '00111001001010110111000100001000001101010111100100010011011000100010111001011110': 'G>a\x0f nCfsW',
 '01111